In [0]:
# Load Silver table
silver_df = spark.table("fraud_catalog.fraud_schema.silver_transactions")

# Preview
silver_df.show(5)
silver_df.printSchema()


+----+--------+---------+-----------+-------------+--------------+----------+--------------+--------------+-------+--------------+
|step|    type|   amount|   nameOrig|oldbalanceOrg|newbalanceOrig|  nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+---------+-----------+-------------+--------------+----------+--------------+--------------+-------+--------------+
|  42| PAYMENT| 14347.57| C169454224|      32702.0|      18354.43|M826816809|           0.0|           0.0|      0|             0|
|  42|CASH_OUT|188336.31| C517661735|          0.0|           0.0|C413194235|     696059.94|     996768.24|      0|             0|
|  42| PAYMENT|  8125.56|C2073018087|       4888.0|           0.0|M475964317|           0.0|           0.0|      0|             0|
|  42|TRANSFER|873638.32| C994720712|      21811.0|           0.0|C726721232|     600098.27|    1473736.59|      0|             0|
|  42|CASH_OUT|257236.72|C1974136200|       9018.0|           0.0|C684221417|      

In [0]:
from pyspark.sql.functions import col, sum as _sum, count, when

# Aggregate metrics for Gold layer
gold_type_df = silver_df.groupBy("type") \
    .agg(
        count("*").alias("total_transactions"),
        _sum("amount").alias("total_amount"),
        _sum(when(col("isFraud") == 1, 1).otherwise(0)).alias("fraud_count"),
        _sum(when(col("isFraud") == 1, col("amount")).otherwise(0)).alias("fraud_amount")
    )

# Save Gold table
gold_type_df.write.format("delta").mode("overwrite").saveAsTable("fraud_catalog.fraud_schema.gold_transactions")

# Preview
gold_type_df.show()


+--------+------------------+--------------------+-----------+-------------------+
|    type|total_transactions|        total_amount|fraud_count|       fraud_amount|
+--------+------------------+--------------------+-----------+-------------------+
| PAYMENT|           2151495|2.809337113836994E10|          0|                0.0|
|TRANSFER|            532909|4.852919872631681E11|       4097|6.067213184010003E9|
| CASH_IN|           1399284|2.363673919124598E11|          0|                0.0|
|CASH_OUT|           2237484|3.944129952244902...|       4100|5.989202243830005E9|
|   DEBIT|             41432| 2.271992212799996E8|          0|                0.0|
+--------+------------------+--------------------+-----------+-------------------+



In [0]:
from pyspark.sql.functions import col, sum as _sum, count, when

# Load Silver table
silver_df = spark.table("fraud_catalog.fraud_schema.silver_transactions")

# Aggregate metrics by transaction type (Gold table)
gold_type_df = silver_df.groupBy("type") \
    .agg(
        count("*").alias("total_transactions"),
        _sum("amount").alias("total_amount"),
        _sum(when(col("isFraud") == 1, 1).otherwise(0)).alias("fraud_count"),
        _sum(when(col("isFraud") == 1, col("amount")).otherwise(0)).alias("fraud_amount")
    )

# Save Gold table
gold_type_df.write.format("delta").mode("overwrite").saveAsTable("fraud_catalog.fraud_schema.gold_transactions")

# Preview
gold_type_df.show()


+--------+------------------+--------------------+-----------+-------------------+
|    type|total_transactions|        total_amount|fraud_count|       fraud_amount|
+--------+------------------+--------------------+-----------+-------------------+
| PAYMENT|           2151495|2.809337113836994E10|          0|                0.0|
|TRANSFER|            532909|4.852919872631681E11|       4097|6.067213184010003E9|
| CASH_IN|           1399284|2.363673919124598E11|          0|                0.0|
|CASH_OUT|           2237484|3.944129952244902...|       4100|5.989202243830005E9|
|   DEBIT|             41432| 2.271992212799996E8|          0|                0.0|
+--------+------------------+--------------------+-----------+-------------------+

